In [1]:
import sys
import csv
import time
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn import datasets
import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

In [8]:
def time_line(pt,st):
    df=pd.read_csv( str(pt) + '/3_final_dataset.csv',header=0,sep=',')
    df_int=pd.read_csv( str(pt) + '/1_target_list.csv',header=0,sep=',')
    #remove duplicates to keep a time of earliest interaction.
    df_click=df_int[df_int.interaction_type==1]
    df_click.drop_duplicates(keep='first')
    df_book=df_int[df_int.interaction_type==2]
    df_book.drop_duplicates(keep='first')
    df_reply=df_int[df_int.interaction_type==3]
    df_reply.drop_duplicates(keep='first')
    df_undisp=df_int[df_int.interaction_type==0]
    
    df_click=df_click.groupby(['user_id','item_id'])['created_at'].apply(lambda x: np.min(x)).reset_index()
    df_book=df_book.groupby(['user_id','item_id'])['created_at'].apply(lambda x: np.min(x)).reset_index()
    df_reply=df_reply.groupby(['user_id','item_id'])['created_at'].apply(lambda x: np.min(x)).reset_index()
    df_undisp=df_undisp.groupby(['user_id','item_id'])['created_at'].apply(lambda x: np.min(x)).reset_index()
    
    df=pd.merge(df,df_click,how='left', on=['user_id','item_id'])
    df=df.rename(columns = {'created_at':'week_click'})
    df=pd.merge(df,df_book,how='left', on=['user_id','item_id'])
    df=df.rename(columns = {'created_at':'week_book'})
    df=pd.merge(df,df_reply,how='left', on=['user_id','item_id'])
    df=df.rename(columns = {'created_at':'week_reply'})
    df=pd.merge(df,df_undisp,how='left', on=['user_id','item_id'])
    df=df.rename(columns = {'created_at':'week_undisp'})
    
    df.week_click.fillna(df.week_undisp, inplace=True)
    df.week_book.fillna(df.week_undisp, inplace=True)
    df.week_reply.fillna(df.week_undisp, inplace=True)
    
    df.set_index(['user_id', 'item_id'], inplace=True)
    
    df.to_csv(str(st)+'/4_time_line.csv')
    return df

In [9]:
time_line("/Users/amirasarbaev/Desktop","/Users/amirasarbaev/Desktop")

,,displayed,click,book,reply,f1,f2,f3,f4,f5,f6,...,discipline_id_y,industry_id_y,country_cat,region_y,employment,active_during_test,week_click,week_book,week_reply,week_undisp
user_id,item_id,,,,,,,,,,,,,,,,,,,,,
151,16291,1,0,0,0,0.1350,0.0650,0.9851,0.0,0.26,0.0739,...,23,15,2,9,1,0,40.0,40.0,40.0,40.0
16874,16291,1,0,0,0,0.0000,0.0000,0.0000,0.0,0.00,0.0043,...,23,15,2,9,1,0,42.0,42.0,42.0,42.0
22589,16291,1,1,0,0,0.1932,0.0879,0.9851,0.0,0.36,0.0847,...,23,15,2,9,1,0,43.0,NaN,NaN,NaN
23514,16291,1,0,0,0,0.1189,0.0534,0.9851,0.0,0.24,0.1072,...,23,15,2,9,1,0,41.0,41.0,41.0,41.0
24608,16291,1,0,0,0,0.1135,0.0678,0.9851,0.0,0.18,0.1089,...,23,15,2,9,1,0,42.0,42.0,42.0,42.0
27885,16291,1,0,0,0,0.0519,0.0350,0.8817,0.0,0.08,0.0348,...,23,15,2,9,1,0,41.0,41.0,41.0,41.0
30556,16291,1,1,0,0,0.2130,0.0934,0.9851,0.0,0.38,0.0666,...,23,15,2,9,1,0,41.0,NaN,NaN,NaN
33704,16291,1,0,0,0,0.1760,0.0849,0.9851,0.0,0.30,0.1171,...,23,15,2,9,1,0,40.0,40.0,40.0,40.0
41458,16291,1,0,0,0,0.0841,0.0400,0.7676,0.0,0.16,0.0585,...,23,15,2,9,1,0,40.0,40.0,40.0,40.0
